In [1]:
import numpy as np
import math
import sys
import ipdb

sys.path.append("../")

sys.path.append("../utils/")
from utils_functions import *

from dd_game import *
from two_stage_player import *
from dfo_player import *
from solo_player import *
from online_player import *

from tqdm import tqdm, trange

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
plt.rcParams['font.size'] = 20
plt.rcParams['axes.linewidth'] = 3

In [2]:
np.random.seed(63)

d_1 = 2  # Dimension of each player's data/strategy
d_2 = 2

epsilon_0 = 0.1
epsilon_1 = 1  # Epsilon sensitivity parameters
epsilon_2 = 1

Sigma_x_p1 = np.eye(d_1)  # Covariance of x^k
Sigma_x_p2 = np.eye(d_2)

sigma_y_p1 = 0.1  # Variance of y^k
sigma_y_p2 = 0.1

beta_p1 = sample_sphere(1,d_1)
beta_p2 = sample_sphere(1,d_2)

# s_1 = sample_sphere(epsilon_1,d_1+d_2)  # Performativity parameters
# mu_p1 = s_1[0:d_1]
# gamma_p1 = s_1[d_1:]
# s_2 = sample_sphere(epsilon_2,d_1+d_2)
# mu_p2 = s_2[0:d_2]
# gamma_p2 = s_2[d_2:]

mu_p1 = sample_sphere(epsilon_0,d_1)
gamma_p1 = sample_sphere(epsilon_1,d_2)
mu_p2 = sample_sphere(epsilon_0,d_2)
gamma_p2 = sample_sphere(epsilon_2,d_1)

p1_data_params = (Sigma_x_p1, sigma_y_p1, beta_p1, mu_p1, gamma_p1)
p1_data_generating_func = sample_from_location_family

p2_data_params = (Sigma_x_p2, sigma_y_p2, beta_p2, mu_p2, gamma_p2)
p2_data_generating_func = sample_from_location_family

N1 = 1
N2 = 1

num_rounds_lst = np.logspace(1.7,5,N2,dtype='int')
# num_rounds_lst = [50, 100, 500, 1000, 5000, 10000, 50000, 100000]
num_alternate_rounds = 100
num_test = 100

In [3]:
p1_risks = np.zeros((N1,N2))
p2_risks = np.zeros((N1,N2))
theta_final_p1 = np.zeros((N1,N2,d_1))
theta_final_p2 = np.zeros((N1,N2,d_2))

delta = 1e-0
eta = 1e-1
num_rounds = num_rounds_lst[-1]

for j in trange(N1):
    player_one = DFOPlayer(delta,eta)
    player_two = DFOPlayer(delta,eta)
    game_dfo = DecisionDependentGame(player_one, player_two,
                                     p1_data_params, p2_data_params,
                                     p1_data_generating_func, p2_data_generating_func)

    for i in range(num_rounds+1):
        game_dfo.theta_p1 = player_one.perturb_theta()
        game_dfo.theta_p2 = player_two.perturb_theta()

        PR_1_oracle, PR_2_oracle = game_dfo.evaluate_closed_perf_risk()

        game_dfo.theta_p1 = player_one.update_theta(PR_1_oracle)
        game_dfo.theta_p2 = player_two.update_theta(PR_2_oracle)

        if i in num_rounds_lst:
            p1_risks[j,np.where(num_rounds_lst==i)] = PR_1_oracle
            p2_risks[j,np.where(num_rounds_lst==i)] = PR_2_oracle

            theta_final_p1[j,np.where(num_rounds_lst==i)] = game_dfo.theta_p1
            theta_final_p2[j,np.where(num_rounds_lst==i)] = game_dfo.theta_p2

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 100.23it/s]


In [4]:
p3_risks = np.zeros((N1,N2))
p4_risks = np.zeros((N1,N2))
theta_final_p3 = np.zeros((N1,N2,d_1))
theta_final_p4 = np.zeros((N1,N2,d_2))

for i in trange(N1):
    for num_rounds in num_rounds_lst:
        player_three = TwoStagePlayer()
        player_four = TwoStagePlayer()
        game = DecisionDependentGame(player_three, player_four,
                                     p1_data_params, p2_data_params,
                                     p1_data_generating_func, p2_data_generating_func,
                                     num_rounds, num_alternate_rounds, num_test)

        game.run_train()
        game.run_post_train_alternating()

        PR3, PR4 = game.evaluate_closed_perf_risk()
        p3_risks[i,np.where(num_rounds_lst==num_rounds)] = PR3
        p4_risks[i,np.where(num_rounds_lst==num_rounds)] = PR4

        theta_final_p3[i,np.where(num_rounds_lst==num_rounds)] = game.theta_p1
        theta_final_p4[i,np.where(num_rounds_lst==num_rounds)] = game.theta_p2

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 18.18it/s]


In [6]:
p5_risks = np.zeros((N1,N2))
p6_risks = np.zeros((N1,N2))
theta_final_p5 = np.zeros((N1,N2,d_1))
theta_final_p6 = np.zeros((N1,N2,d_2))

nu = 1e-2
eta = 1e-1
n = 100
B = 0.1
R = 5
num_rounds = num_rounds_lst[-1]

for k in trange(N1):
    player_five = OnlinePlayer(d_1, d_1+d_2, d_1, eta, nu, n, B, R)
    player_six = OnlinePlayer(d_2, d_1+d_2, d_2, eta, nu, n, B, R)
    game_online = DecisionDependentGame(player_five, player_six,
                                        p1_data_params, p2_data_params,
                                        p1_data_generating_func, p2_data_generating_func,
                                        num_test = 1000)
    l=0
    for i in trange(math.ceil(num_rounds/(n))+1):

        theta_5, u_5k = player_five.perturb_theta()
        theta_6, u_6k = player_six.perturb_theta()
        z_5k = []
        z_6k = []
        for j in range(n):
            game_online.theta_p1 = theta_5+(u_5k[:,j]).reshape((-1,1))
            game_online.theta_p2 = theta_5+(u_6k[:,j]).reshape((-1,1))

            z_5k.append(game_online.oracle_z1())
            z_6k.append(game_online.oracle_z2())
        print(z_5k)
        q_5k = player_five.compute_aux_q(z_5k, theta_6, u_6k)
        q_6k = player_six.compute_aux_q(z_6k, theta_5, u_5k)
        p_5 = player_five.update_p()
        p_6 = player_six.update_p()
        grad_5 = game_online.oracle_grad1(p_5, q_5k)
        grad_6 = game_online.oracle_grad2(p_6, q_6k)
        game_online.theta_p1 = player_five.update_theta(grad_5)
        game_online.theta_p2 = player_six.update_theta(grad_6)
        if i*n >= num_rounds_lst[len(p5_risks)]:
            l+=1
            p5_risks[k,l] = game_online.evaluate_perf_risk_p1()
            p6_risks[k,l] = game_online.evaluate_perf_risk_p2()

            theta_final_p5[k,l] = game_online.theta_p1
            theta_final_p6[k,l] = game_online.theta_p2

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

[(array([ 0.43665457, -0.6517068 ]), array([-0.8536443])), (array([ 0.57628284, -0.57845744]), array([-0.5956369])), (array([ 1.41113661, -1.58734315]), array([-1.75858597])), (array([0.49735829, 0.19120459]), array([-0.41239107])), (array([-1.88434581,  0.4685078 ]), array([1.8319143])), (array([-0.03699006,  1.86897481]), array([1.09589755])), (array([-0.91905479, -0.95892077]), array([0.37809645])), (array([0.178808  , 2.49971541]), array([0.83265518])), (array([-0.55881958,  1.09663642]), array([0.94619186])), (array([-0.30368925, -0.65171502]), array([0.03361044])), (array([-1.49689515, -0.40885839]), array([1.34427781])), (array([0.60394524, 0.86943707]), array([-0.10312625])), (array([ 1.46416319, -0.12601775]), array([-1.42764118])), (array([ 0.84333738, -1.2730154 ]), array([-1.4929027])), (array([2.04374775, 0.77749639]), array([-1.53760477])), (array([-0.06795298, -0.6871831 ]), array([-0.39529341])), (array([1.72767566, 0.98362908]), array([-1.11618977])), (array([1.2219573

ValueError: operands could not be broadcast together with shapes (100,2) (1,100) 

In [ ]:
p1_risks[np.isinf(p1_risks)]=np.nan
p2_risks[np.isinf(p2_risks)]=np.nan
p3_risks[np.isinf(p3_risks)]=np.nan
p4_risks[np.isinf(p4_risks)]=np.nan

theta_PO_1, theta_PO_2 = game.solve_nash()
theta_SO_1, theta_SO_2 = game.solve_social_opt()

game.theta_p1, game.theta_p2 = game.solve_nash()
PO_1, PO_2 = game.evaluate_closed_perf_risk()
print(f'Player 1 PO = {PO_1.round(3)}\tPlayer 2 PO = {PO_2.round(3)}')

game.theta_p1, game.theta_p2 = game.solve_social_opt()
SO_1, SO_2 = game.evaluate_closed_perf_risk()
print(f'Player 1 SO = {SO_1.round(3)}\tPlayer 2 SO = {SO_2.round(3)}')

In [ ]:
plt.rcParams['font.size'] = 28
plt.rcParams['axes.linewidth'] = 4
plt.figure(figsize=(10,10))
# plt.subplot(2,1,1)
# plt.semilogx(num_rounds_lst, [PO_1 for i in num_rounds_lst],
#              '-', color='k', label='P1 Performative Optimum')
# plt.semilogx(num_rounds_lst, [PO_2 for i in num_rounds_lst],
#              '-', color='k', label='P2 Performative Optimum') 

# plt.semilogx(num_rounds_lst, p1_risks,
#              '--', color='#5553E6', label='P1 Solo Performative Risk')
# plt.semilogx(num_rounds_lst, p2_risks,
#              '--', color='#E07110', label='P2 2S Performative Risk')

# plt.semilogx(num_rounds_lst, p3_risks,
#              '-', color='#5553E6', label='P1 2S Performative Risk')
# plt.semilogx(num_rounds_lst, p4_risks,
#              '-', color='#E07110', label='P2 2S Performative Risk')
# plt.ylabel(r"$PR(\theta)$")
# plt.legend(fontsize=14)

# plt.subplot(2,1,2)
plt.loglog(num_rounds_lst, [PO_1 for i in num_rounds_lst],
           '-', color='k', label='PO', lw=3)
plt.loglog(num_rounds_lst, [PO_2 for i in num_rounds_lst],
           '-', color='k', lw=4)

# plt.loglog(num_rounds_lst, p1_risks,
#            '--', color='#5553E6', label='P1 Solo Performative Risk')
# plt.loglog(num_rounds_lst, p2_risks,
#            '--', color='#E07110', label='P2 2S Performative Risk')

plt.loglog(num_rounds_lst, np.nanmean(p3_risks,axis=0),
           '-', color='#5553E6', label='P1 Algorithm 1', lw=4, marker='o',markersize=15)
plt.fill_between(num_rounds_lst, np.sort(p3_risks,axis=0)[math.floor(N1*0.34)],
                 np.sort(p3_risks,axis=0)[-math.ceil(N1*0.34)],
                 alpha=0.2, color='#5553E6')
plt.loglog(num_rounds_lst, np.nanmean(p4_risks,axis=0),
           '-', color='#E07110', label='P2 Algorithm 1', lw=4, marker='o',markersize=15)
plt.fill_between(num_rounds_lst, np.sort(p4_risks,axis=0)[math.floor(N1*0.34)],
                 np.sort(p4_risks,axis=0)[-math.ceil(N1*0.34)],
                 alpha=0.2, color='#E07110')

plt.loglog(num_rounds_lst, np.nanmean(p1_risks,axis=0),
           '--', color='#5553E6', label='P1 DFO', lw=4, marker='P',markersize=15)
plt.fill_between(num_rounds_lst, np.sort(p1_risks,axis=0)[math.floor(N1*0.34)],
                 np.sort(p1_risks,axis=0)[-math.ceil(N1*0.34)],
                 alpha=0.2, color='#5553E6')
plt.loglog(num_rounds_lst, np.nanmean(p2_risks,axis=0),
           '--', color='#E07110', label='P2 DFO', lw=4, marker='P',markersize=15)
plt.fill_between(num_rounds_lst, np.sort(p2_risks,axis=0)[math.floor(N1*0.34)],
                 np.sort(p2_risks,axis=0)[-math.ceil(N1*0.34)],
                 alpha=0.2, color='#E07110')

plt.ylabel("Performative Risk")
plt.xlabel("Samples")
plt.legend(loc='upper right',fontsize=24)
plt.ylim(0.09,1.3)
plt.tight_layout()
plt.savefig('../figures/dfo_vs_2stage_convergence.pdf', format='pdf')
plt.show()

In [ ]:
plt.rcParams['font.size'] = 24
plt.rcParams['axes.linewidth'] = 3
plt.figure(figsize=(10,10))

plt.loglog(num_rounds_lst, [PO_1 for i in num_rounds_lst],
           '-', color='k', label='PO', lw=3)
plt.loglog(num_rounds_lst, [PO_2 for i in num_rounds_lst],
           '-', color='k', lw=3)

plt.loglog(num_rounds_lst, np.nanmean(p3_risks,axis=0),
           '-', color='#5553E6', label='P1 Algorithm 1', lw=3)
plt.fill_between(num_rounds_lst, np.nanmean(p3_risks,axis=0)-2*np.sqrt(np.nanvar(p3_risks,axis=0)),
                 np.nanmean(p3_risks,axis=0)+2*np.sqrt(np.nanvar(p3_risks,axis=0)),
                 alpha=0.2, color='#5553E6')
plt.loglog(num_rounds_lst, np.nanmean(p4_risks,axis=0),
           '-', color='#E07110', label='P2 Algorithm 1', lw=3)
plt.fill_between(num_rounds_lst, np.nanmean(p4_risks,axis=0)-2*np.sqrt(np.nanvar(p4_risks,axis=0)),
                 np.nanmean(p4_risks,axis=0)+2*np.sqrt(np.nanvar(p4_risks,axis=0)),
                 alpha=0.2, color='#E07110')

plt.loglog(num_rounds_lst, np.nanmean(p1_risks,axis=0),
           '--', color='#5553E6', label='P1 DFO', lw=3)
plt.fill_between(num_rounds_lst, np.nanmean(p1_risks,axis=0)-2*np.sqrt(np.nanvar(p1_risks,axis=0)),
                 np.nanmean(p1_risks,axis=0)+2*np.sqrt(np.nanvar(p1_risks,axis=0)),
                 alpha=0.2, color='#5553E6')
plt.loglog(num_rounds_lst, np.nanmean(p2_risks,axis=0),
           '--', color='#E07110', label='P2 DFO', lw=3)
plt.fill_between(num_rounds_lst, np.nanmean(p2_risks,axis=0)-2*np.sqrt(np.nanvar(p2_risks,axis=0)),
                 np.nanmean(p2_risks,axis=0)+2*np.sqrt(np.nanvar(p2_risks,axis=0)),
                 alpha=0.2, color='#E07110')

plt.ylabel("Performative Risk")
plt.xlabel("Samples")
plt.legend(loc='best',fontsize=24)
plt.savefig('../figures/dfo_vs_2stage_convergence_var.pdf', format='pdf')
plt.tight_layout()
plt.show()

In [ ]:
np.mean(np.linalg.norm(theta_final_p1-theta_PO_1,axis=2),axis=0)

In [ ]:
theta_final_p1